In [1]:
## Môi trường lập trình
from google.colab import drive
drive.mount('/content/gdrive')
folder = '/content/gdrive/My Drive/Năm IV/Big Data/Crawl'

Mounted at /content/gdrive


In [2]:
import requests
import pandas as pd
import time
import random
from tqdm import tqdm

In [8]:
cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{"access_token":"8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY","expires_in":157680000,"expires_at":1763654224277,"guest_token":"8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY"}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj4bc.1k.11.2l',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlrj2q9.3.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605977607.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605977607.0',
    '_ga': 'GA1.2.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzkzizhzkzlznzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkzlzjzgzjzdzizizdzizlzjzmzqzkzlzjzgzjzdzizlzjzmzqzkzlzjzgzjzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605977598%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjAbsoluteSessionInProgress': '0',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605978058486',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    'cto_bundle': '7L6ha19NVXNkQmJ6aEVLcXNqbHdjcVZoQ0kzTUZwcEMyNCUyRm5nV3A2SThuOGxTRjI4Wlk1NU9xRnBEOG9tUjd2ekhyZEQxeE9qaVQ4MnFpbiUyRllGd2JiQUpTMW94MlNsTnYxd3dOYWtRcXhGdDNxSjdBVmNxU0FnUSUyQjlWYjhqTUtLdVl2cTBheWFvS0ZnY2pLdlpWRlEyUFF0Y1ElM0QlM0Q',
    'TIKI_RECENTLYVIEWED': '58259141',
    '_ants_event_his': '%7B%22action%22%3A%22view%22%2C%22time%22%3A1605977607258%7D',
    '_hjIncludedInPageviewSample': '1',
    '_hjIncludedInSessionSample': '1',
    '_gat': '1',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,en-US;q=0.9,en;q=0.8',
    'Referer': 'https://tiki.vn/dien-thoai-samsung-galaxy-m31-128gb-6gb-hang-chinh-hang-p58259141.html',
    'x-guest-token': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = {
    'product_id': '58259141',
    'sort': 'score|desc,id|desc,stars|all',
    'page': '1',
    'limit': '20',
    'include': 'comments'
}

In [9]:
def comment_parser(json):
    d = dict()
    d['id'] = json.get(id)
    d['title'] = json.get('title')
    d['content'] = json.get('content')
    #d['thank_count'] = json.get('thank_count')
    #d['customer_id']  = json.get('customer_id')
    d['rating'] = json.get('rating')
    #d['created_at'] = json.get('created_at')
    #d['customer_name'] = json.get('created_by').get('name')
    #d['purchased_at'] = json.get('created_by').get('purchased_at')
    return d

In [10]:
file_path = '/content/gdrive/My Drive/Năm IV/Big Data/Crawl/product_id_ncds.csv'
df_id = pd.read_csv(file_path)
p_ids = df_id.id.to_list()
result = []
for pid in tqdm(p_ids, total=len(p_ids)):
    params['product_id'] = pid
    print('Crawl comment for product {}'.format(pid))
    for i in range(1,51):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/v2/reviews', headers=headers, params=params, cookies=cookies)
        if response.status_code == 200:
            print('Crawl comment page {} success!!!'.format(i))
            for comment in response.json().get('data'):
                result.append(comment_parser(comment))
df_comment = pd.DataFrame(result)
df_comment.to_csv('/content/gdrive/My Drive/Năm IV/Big Data/Crawl/comments_data_ncds.csv', index=False)

  0%|          | 0/332 [00:00<?, ?it/s]

Crawl comment for product 271046571
Crawl comment page 1 success!!!
Crawl comment page 2 success!!!
Crawl comment page 3 success!!!
Crawl comment page 4 success!!!
Crawl comment page 5 success!!!
Crawl comment page 6 success!!!
Crawl comment page 7 success!!!
Crawl comment page 8 success!!!
Crawl comment page 9 success!!!
Crawl comment page 10 success!!!
Crawl comment page 11 success!!!
Crawl comment page 12 success!!!
Crawl comment page 13 success!!!
Crawl comment page 14 success!!!
Crawl comment page 15 success!!!
Crawl comment page 16 success!!!
Crawl comment page 17 success!!!
Crawl comment page 18 success!!!
Crawl comment page 19 success!!!
Crawl comment page 20 success!!!
Crawl comment page 21 success!!!
Crawl comment page 22 success!!!
Crawl comment page 23 success!!!
Crawl comment page 24 success!!!
Crawl comment page 25 success!!!
Crawl comment page 26 success!!!
Crawl comment page 27 success!!!
Crawl comment page 28 success!!!
Crawl comment page 29 success!!!
Crawl comment pa

  0%|          | 1/332 [00:24<2:16:04, 24.67s/it]

Crawl comment page 50 success!!!
Crawl comment for product 273947792
Crawl comment page 1 success!!!
Crawl comment page 2 success!!!
Crawl comment page 3 success!!!
Crawl comment page 4 success!!!
Crawl comment page 5 success!!!
Crawl comment page 6 success!!!
Crawl comment page 7 success!!!
Crawl comment page 8 success!!!
Crawl comment page 9 success!!!
Crawl comment page 10 success!!!
Crawl comment page 11 success!!!
Crawl comment page 12 success!!!
Crawl comment page 13 success!!!
Crawl comment page 14 success!!!
Crawl comment page 15 success!!!
Crawl comment page 16 success!!!
Crawl comment page 17 success!!!
Crawl comment page 18 success!!!
Crawl comment page 19 success!!!
Crawl comment page 20 success!!!
Crawl comment page 21 success!!!
Crawl comment page 22 success!!!
Crawl comment page 23 success!!!
Crawl comment page 24 success!!!
Crawl comment page 25 success!!!
Crawl comment page 26 success!!!
Crawl comment page 27 success!!!
Crawl comment page 28 success!!!
Crawl comment pa

  1%|          | 2/332 [00:41<1:48:58, 19.81s/it]

...

Crawl comment page 50 success!!!
Crawl comment for product 273244775
Crawl comment page 1 success!!!
Crawl comment page 2 success!!!
Crawl comment page 3 success!!!
Crawl comment page 4 success!!!
Crawl comment page 5 success!!!
Crawl comment page 6 success!!!
Crawl comment page 7 success!!!
Crawl comment page 8 success!!!
Crawl comment page 9 success!!!
Crawl comment page 10 success!!!
Crawl comment page 11 success!!!
Crawl comment page 12 success!!!
Crawl comment page 13 success!!!
Crawl comment page 14 success!!!
Crawl comment page 15 success!!!
Crawl comment page 16 success!!!
Crawl comment page 17 success!!!
Crawl comment page 18 success!!!
Crawl comment page 19 success!!!
Crawl comment page 20 success!!!
Crawl comment page 21 success!!!
Crawl comment page 22 success!!!
Crawl comment page 23 success!!!
Crawl comment page 24 success!!!
Crawl comment page 25 success!!!
Crawl comment page 26 success!!!
Crawl comment page 27 success!!!
Crawl comment page 28 success!!!
Crawl comment pa

100%|█████████▉| 331/332 [1:44:24<00:17, 17.83s/it]

Crawl comment page 50 success!!!
Crawl comment for product 273357326
Crawl comment page 1 success!!!
Crawl comment page 2 success!!!
Crawl comment page 3 success!!!
Crawl comment page 4 success!!!
Crawl comment page 5 success!!!
Crawl comment page 6 success!!!
Crawl comment page 7 success!!!
Crawl comment page 8 success!!!
Crawl comment page 9 success!!!
Crawl comment page 10 success!!!
Crawl comment page 11 success!!!
Crawl comment page 12 success!!!
Crawl comment page 13 success!!!
Crawl comment page 14 success!!!
Crawl comment page 15 success!!!
Crawl comment page 16 success!!!
Crawl comment page 17 success!!!
Crawl comment page 18 success!!!
Crawl comment page 19 success!!!
Crawl comment page 20 success!!!
Crawl comment page 21 success!!!
Crawl comment page 22 success!!!
Crawl comment page 23 success!!!
Crawl comment page 24 success!!!
Crawl comment page 25 success!!!
Crawl comment page 26 success!!!
Crawl comment page 27 success!!!
Crawl comment page 28 success!!!
Crawl comment pa

100%|██████████| 332/332 [1:44:43<00:00, 18.93s/it]

Crawl comment page 50 success!!!
